We'll need the Python Image Library descendant PILLOW. Pathlib and OS libraries are also very helpful. 

In [7]:
from PIL import Image, ImageFilter
import pathlib
import os

Let's gather all our categories (data types) and places (maps).

In [8]:
ps_watercolor_dir = pathlib.Path("./photoshopped/")
masks = list(ps_watercolor_dir.glob("*.png"))

categories = []
places = []

for mask in masks:
    s = mask.stem.split("_")
    category, place = s[0], s[1]
    if category not in categories:
        categories.append(category)
    if place not in places:
        places.append(place)

print(categories)
print(places)
    

['beach', 'buildings', 'farms', 'forest', 'grass', 'heath', 'highways', 'industrial', 'land', 'local-roads', 'major-roads', 'metro', 'parking', 'parks', 'pedestrian', 'train', 'trams', 'water']
['Bossche Broek', 'Danska Falls', 'Den Bosch', 'Engelen', 'Halmstad', 'Maas Gement', 'Pozzuoli', 'Procida', 'Zaltbommel', 'Bleyenbeek', 'Camden', 'Komosse Naturreservat', 'Mullsjön', 'Nationaal park Söderåsen', 'Pompeii', 'Rotterdam', 'Tenhultasjön', 'Utrecht Science Park', 'Vomero', 'Wandelpark Tegelen', 'Copenhagen', 'Moerputten', 'Nationaal Park De Meinweg', 'Nationaal Park Utrechtse Heulvelrug', 'Nationaal Park Veluwezoom', 'Lyngemadssjön', 'Mariesholmsskogen']


Gather all the filenames for the watercolor textures.

In [9]:
watercategories_dir = pathlib.Path(r"U:\Photos_2023\Wildlife2023\Watercolors\categorized")
watercolors = list(watercategories_dir.glob("*.png"))
watercategories = [x.stem for x in watercolors]
print(watercolors)
print(watercategories)

[WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/beach.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/buildings.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/farms.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/forest.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/grass.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/heath.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/industrial.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/land.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/parking.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/parks.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/pedestrian.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Watercolors/categorized/roads.png'), WindowsPath('U:/Photos_2023/Wildlife2023/Waterc

Some data types will use the same watercolor texture, so I need to map the data types to their shared texture.

In [10]:
watercolor_mapping = {}
needs_intervention = []

for category in categories:
    if category in watercategories:
        watercolor_mapping[category] = category
    else:
        needs_intervention.append(category)

watercolor_mapping["train"] = "trains"
watercolor_mapping["metro"] = "trains"
watercolor_mapping["trams"] = "trains"
watercolor_mapping["highways"] = "roads"
watercolor_mapping["major-roads"] = "roads"
watercolor_mapping["local-roads"] = "roads"

print(watercolor_mapping)
print([x for x in needs_intervention if x not in watercolor_mapping.keys()])

{'beach': 'beach', 'buildings': 'buildings', 'farms': 'farms', 'forest': 'forest', 'grass': 'grass', 'heath': 'heath', 'industrial': 'industrial', 'land': 'land', 'parking': 'parking', 'parks': 'parks', 'pedestrian': 'pedestrian', 'water': 'water', 'train': 'trains', 'metro': 'trains', 'trams': 'trains', 'highways': 'roads', 'major-roads': 'roads', 'local-roads': 'roads'}
[]


Apply the masks to the textures, randomly rotate the textures so they don't look exactly the same in every map. 

In [11]:
from random import choice
import PIL
from IPython.display import display
output_built = pathlib.Path(r"U:\Photos_2023\Wildlife2023\built_images")

c = 1
for mask in masks[:]:
    s = mask.stem.split("_")
    category, place = s[0], s[1]
    print(f"{category} -- {place} -- {c} of {len(masks)}")
    outpath = output_built/f"{category}_{place}.png"
    if os.path.exists(outpath) == False:
        print("\tProcessing")
        mask_img = Image.open(mask)
        watercolor_texture = watercategories_dir / f"{watercolor_mapping[category]}.png"
        texture_img = Image.open(watercolor_texture)
        texture_img = texture_img.convert(mode="RGB")
        rotation = choice([0, 90, 180, 270])
        texture_img = texture_img.rotate(rotation, PIL.Image.NEAREST, expand = 1)
        texture_img.putalpha(mask_img)
        texture_img.save(outpath)
    else:
        print("\tEXISTS")
    c+=1

beach -- Bossche Broek -- 1 of 286
	EXISTS
beach -- Danska Falls -- 2 of 286
	EXISTS
beach -- Den Bosch -- 3 of 286
	EXISTS
beach -- Engelen -- 4 of 286
	EXISTS
beach -- Halmstad -- 5 of 286
	EXISTS
beach -- Maas Gement -- 6 of 286
	EXISTS
beach -- Pozzuoli -- 7 of 286
	EXISTS
beach -- Procida -- 8 of 286
	EXISTS
beach -- Zaltbommel -- 9 of 286
	EXISTS
buildings -- Bleyenbeek -- 10 of 286
	EXISTS
buildings -- Bossche Broek -- 11 of 286
	EXISTS
buildings -- Camden -- 12 of 286
	EXISTS
buildings -- Danska Falls -- 13 of 286
	EXISTS
buildings -- Den Bosch -- 14 of 286
	EXISTS
buildings -- Engelen -- 15 of 286
	EXISTS
buildings -- Halmstad -- 16 of 286
	EXISTS
buildings -- Komosse Naturreservat -- 17 of 286
	EXISTS
buildings -- Maas Gement -- 18 of 286
	EXISTS
buildings -- Mullsjön -- 19 of 286
	EXISTS
buildings -- Nationaal park Söderåsen -- 20 of 286
	EXISTS
buildings -- Pompeii -- 21 of 286
	EXISTS
buildings -- Pozzuoli -- 22 of 286
	EXISTS
buildings -- Procida -- 23 of 286
	EXISTS
buil

Sometimes the land texture, being a complete white mask produced errors, so just randomly rotate and rename land texture and export to the GIS folder. 

In [6]:
for place in places:
    outpath = output_built / f"land_{place}.png"
    if os.path.exists(outpath) == False:
        texture_img = Image.open(r"U:\Photos_2023\Wildlife2023\Watercolors\categorized\land.png")
        texture_img = texture_img.convert(mode="RGB")
        rotation = choice([0, 90, 180, 270])
        texture_img = texture_img.rotate(rotation, PIL.Image.NEAREST, expand = 1)
        texture_img.save(outpath)